In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Stemmer
from stop_words import ESP_STOP_WORDS as BASE_STOP_WORDS, ENG_STOP_WORDS
from html.parser import HTMLParser
import Stemmer
from datasketch import MinHash
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix
import heapq

from rutas import *

%matplotlib inline

In [32]:
df2 = pd.read_csv(RUTA_AVISOS_DETALLE)

In [33]:
cantidad_de_palabras_que_quiero = 500
porcetaje=0.01
similitud=0.3

In [34]:
#from descripcion_parser import parse

In [35]:
stopswords="a acá ahí ajena ajeno ajenos ajenas al algo algún algúna algúno algúnos algúnas allá allí ambos ante antes aquel aquella aquello aquellos aquellas aquí arriba así atrás aun aunque bajo bastante bien cabe cada casi cierto cierta ciertos ciertas como con conmigo conseguimos conseguir consigo consigue consiguen consigues contigo contra cual cuales cualquier cualquiera cualquieras cuancuando cuanto cuantas cuanta cuantos de dejar del demás demasiada demasiadas demasiado demasiados dentro desde donde dos e el él ella ellas ello ellos empleáis emplean emplear empleas empleo en encima entonces entre era eras eramos eran eres es esa ese eso esos esas esta estas estaba estado estáis estamos están estar este esto estos estas estoy etc fin fue fueron fui fuimos gueno ha hace haces hacéis hacemos hacen hacer hacia hago hasta incluso intenta intentas intentáis intentamos intentan intentar intento ir jamás junto juntos la lo las los largo más me menos mi mis mía mías mientras mío míos misma mismo mismos mismas modo mucha muchas muchísima muchísimos muchísimo muchísimas mucho muchos muy nada ni ningún ningúna ningúnas ningúno ningúnos no nos nosotras nosotros nuestra nuestro nuestros nuestras nunca o os otra otro otras otros para parecer pero poca poco pocas pocos podéis podemos poder podría podrías podríais podríamos podrían por por qué porque primero puede pueden puedo pues que qué querer quién quiénes quienesquiera quienquiera quizá quizás sabe sabes saben sabéis sabemos saber se según ser si sí siempre siendo sin sino so sobre sois solamente solo sólo somos soy sr sra sres sta su sus suyo suyos suya suyas tal tales también tampoco tan tanta tanto tantos tantas te tenéis tenemos tener tengo ti tiempo tiene tienen todo todos toda todas tomar trabaja trabajo trabajáis trabajamos trabajan trabajar trabajas tras tú tu tus tuya tuyas tuyo tuyos último ultimo un una unas uno unos usa usas usáis usamos usan usar uso usted ustedes va van vais valor vamos varias varios vaya verdadera vosotras vosotros voy vuestra vuestros vuestro vuestras y ya yo como donde requisitos requisito buscamos importante buenas muy grandes beneficios encontramos busqueda zona "
stopswords=stopswords.split()

In [36]:
stopswords = set([ 
 'preferentemente'
]).union(BASE_STOP_WORDS.union(ENG_STOP_WORDS).union(stopswords))

In [37]:
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed =[]
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_html_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def foldear_simbolos(cadena):
    ''' 
    Elimina los acentos de una cadena en minúsculas. 
    Ej: descripción -> descripcion
    '''
    acentos = {'á':'a', 'é':'e', 'í':'i', 'ó':'o', 'ú':'u', 'ü':'u'}
    cadena = list(cadena)
    for i in range(len(cadena)):
        if not cadena[i].isalpha():
            cadena[i] = ' '
        else:
            cadena[i] = acentos.get(cadena[i], cadena[i])
    
    return ''.join(cadena)

def es_palabra_inutil(palabra):
    '''
    Devuelve True si palabra es una palabra que no aporta
    contenido.
    '''
    return palabra in set(stopswords)

def parse(descripcion):
    '''
    Normaliza una descripción de aviso del set de datos de Navent.
    Elimina tags HTML, pasa a minúsculas y la convierte a tokens.
    '''
    resultado = []
    
    for palabra in strip_html_tags(descripcion).lower().split():
        if es_palabra_inutil(palabra):
            continue
            
        resultado.extend(foldear_simbolos(palabra).split())
    
    return ' '.join(resultado)

In [38]:
words={}
def total_freq(description):
    #it create a dictionary with the following format (word : #appearances in the sum of descriptions)
    words_processed=[]
    for word in description.split():
        if word not in words_processed:
            words[word] = words.get(word, 0) + 1
            words_processed.append(word)
    return description

factor = df2["descripcion"].count()*porcetaje #aca va el porcentaje que se quiere tener

def remove_less_frequent_words(description):
    # we will remove all the words which appear in less than 1% of the descrptions
    result = []
    for word in description.lower().split():
        if words[word] < factor:
            continue
            
        result.append(word)
    
    return ' '.join(result)

In [39]:
len(df2)

25288

In [40]:
df2["descripcion"]=df2["descripcion"].map(parse)

In [41]:
stemmer = Stemmer.Stemmer('spanish')
raices={}

def conseguir_raices(descripcion):
    
    for palabra in descripcion.lower().split():
        if palabra in raices:
            continue
        p=stemmer.stemWords([palabra])
        raices[palabra]=p[0]
    return descripcion
    
def reempazar_por_raiz(description):
    
    result = []
    for word in description.lower().split():
        word=raices[word]    
        result.append(word)
    
    return ' '.join(result)
    

In [42]:
df2["descripcion"]=df2["descripcion"].map(conseguir_raices)

In [43]:
df2["descripcion"]=df2["descripcion"].map(reempazar_por_raiz)

In [44]:
df2["descripcion"]=df2["descripcion"].map(total_freq)

In [45]:
df2["descripcion"] = df2["descripcion"].map(remove_less_frequent_words)

In [46]:
palabras = set()

def f(x):
    global palabras
    palabras = palabras.union(set(x.split()))

df2['descripcion'].map(f);

In [47]:
df2[df2['descripcion'].map(lambda x: len(x.split())) == 0]

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
1621,1112410726,1,testestestestestestest social boost,,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Organización y Métodos,TEST MDAN2
2080,1112282436,1,Taquillero testest,,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Oficios y Profesiones,TEST MDAN2
3831,1112423971,1,Testestestestestestest,,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Tecnologia / Sistemas,TEST MDAN2
4059,1112425243,1,manicura esculpidora de uñas,,Capital Federal,NaN,"av. Rivadavia 18391, moron",Full-time,Senior / Semi-Senior,Estética y Cuidado Personal,Clubdelcorte
4376,1112361790,1,Esto es un aviso de prueba,,Capital Federal,NaN,rondeau 2664,Full-time,Senior / Semi-Senior,Tecnologia / Sistemas,TEST MDAN2
4753,1112429613,1,.,,Gran Buenos Aires,NaN,NaN,Fines de Semana,Otro,Distribución,TEST MDAN2
6200,1112373687,1,Domestic Bonsai busca vendedor/a para E-comer...,,Capital Federal,NaN,Luis Viale 1045,Full-time,Senior / Semi-Senior,Ventas,domestic bonsai sa
7602,1112447730,1,boosteo,,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Organización y Métodos,BUMERAN.COM ARGENTINA
7603,1112447747,1,boosteo,,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Administración,BUMERAN.COM ARGENTINA
9734,1112397889,1,Asistente de Logística,,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Logística,comprando en grupo


In [48]:
df2 = df2.drop(axis=1, labels=['idpais', 'nombre_zona', 'ciudad', 'mapacalle', 'titulo', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'denominacion_empresa'])

In [49]:
for p in palabras:
    if p != 'descripcion':
        df2[p] = 0

In [50]:
df2.head()

In [52]:
def baggear_of_words(reg):
    for palabra in reg['descripcion'].split():
        if palabra != 'descripcion':
            reg[palabra] = 1
    
    return reg

df2 = df2.apply(baggear_of_words, axis=1)

In [60]:
df_km = df2.drop(axis=1, labels=['idaviso', 'descripcion'])

In [77]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_jobs=-1, verbose=1, n_clusters=500)

In [78]:
kmeans.fit(df_km)

Initialization complete
Initialization complete
Initialization complete
Initialization complete
start iteration
done sorting
end inner loop
start iteration
done sorting
end inner loop
Iteration 0, inertia 829230.3099217208
start iteration
done sorting
Iteration 0, inertia 827887.542080104
start iteration
done sorting
start iteration
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 0, inertia 828600.8101765327
start iteration
done sorting
Iteration 0, inertia 827934.9450247864
start iteration
done sorting
end inner loop
end inner loop
Iteration 1, inertia 794103.6776268202
start iteration
done sorting
Iteration 1, inertia 790441.8042586208
start iteration
done sorting
end inner loop
end inner loop
end inner loop
Iteration 1, inertia 792753.2799827363
start iteration
done sorting
end inner loop
Iteration 1, inertia 792750.6532442289
start iteration
done sorting
Iteration 2, inertia 785971.725189762
start iteration
done sorting
Iteration 2, inertia 782039.

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=500, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=1)

In [79]:
preds = kmeans.predict(df_km)

In [83]:
merged = pd.merge(pd.DataFrame(preds), df2[['idaviso', 'descripcion']], left_index=True, right_index=True)
merged[merged[0] == 0]

In [84]:
merged = pd.merge(pd.read_csv(RUTA_AVISOS_DETALLE), merged, on='idaviso')

In [89]:
contador = 0
def mostrar(x):
    global contador
    if contador < 5:
        print(x)
        contador += 1
merged[merged[0] == 12]['descripcion_x'].map(mostrar)

<p><em><strong>DIA</strong>, Distribuidora Internacional de Alimentación, es una cadena de distribución nacida en España en 1979 con la apertura de su primera tienda en Madrid. Consolidado como líder y referente en el sector de la distribución, el Grupo DIA es hoy una reconocida multinacional española que está presente en 5 países: Argentina, Brasil, España, Portugal y China.</em></p><p><em>Supermercados Dia, Importante Multinacional Líder en Retail, seleccionará un</em> <strong>Controlador de Stock Nocturno</strong><em>.</em></p><p><em>Entre sus tareas deberá:</em></p><ul><li><em>Realizar inventarios en la red de sucursales y en los centros de distribución.</em></li><li><em>Informar la pérdida conocida y desconocida relevada en cada sucursal. </em></li><li><em>Realizar el conteo de la mercadería para controlar el stock e informar irregularidades de la misma. </em></li></ul><p>Requisitos:</p><p><em>Orientamos nuestra búsqueda a aquellos perfiles que tengan sus estudios secundarios comp

189      None
338      None
413      None
433      None
574      None
1344     None
1362     None
1496     None
1675     None
1826     None
1836     None
1960     None
2293     None
2404     None
2429     None
2465     None
2579     None
2704     None
2866     None
3054     None
3057     None
3578     None
3625     None
3802     None
4060     None
4209     None
4310     None
4577     None
4752     None
4859     None
         ... 
20506    None
20868    None
20931    None
20947    None
20998    None
21342    None
21468    None
21512    None
21547    None
21615    None
21803    None
22188    None
22217    None
22250    None
22476    None
22925    None
23029    None
23412    None
23517    None
23561    None
23602    None
23651    None
23821    None
23881    None
24486    None
24610    None
24961    None
25116    None
25186    None
25201    None
Name: descripcion_x, Length: 182, dtype: object

In [22]:
# sudo pip3 install datasketch -U
#te da lasdescripciones que tengan semejanza mayor a 0.3 (este es un buen factor)

data1=df2["descripcion"][43]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> similitud:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)  
print("ok")

Estimated Jaccard for data1 and data2 is 1.0
busqued anal credit jr sector entid bancari posicion respons ser analisis evaluacion propuest credit participacion conjunt credit administracion carter requisit postulat si sos profesional estudi avanz carrer cienci econom licenci administracion ingenier ten actitud proactiv manej relacion interpersonal equip orientacion result cont manej paquet offic valor conoc segment experient laboral analisis balanc riesg credit benefici condicion contratacion lun viern jorn tim trabaj microcentr empres ofrec condicion contratacion
Estimated Jaccard for data1 and data2 is 0.3046875
busqued un estudi carrer cienci econom desempeñ anal junior administracion finanz principal tar ser participacion proces pag proveedor emision valor fisic y o electron confeccion legaj pag proveedor cambi gast estructur confeccion administracion fond fij pag viaj otras busqued orient person agil dinam capac organizacion relacion interpersonal aprend desarroll are ser excluyen

KeyboardInterrupt: 

In [50]:
#tf = TfidfVectorizer(input='/home/iglesias/test', analyzer='word', ngram_range=(1,6),
#min_df = 0, stop_words = 'english', sublinear_tf=True)
tf = TfidfVectorizer() 
tfidf_matrix =  tf.fit_transform(df2["descripcion"])


feature_names = tf.get_feature_names()

"""doc =0
feature_index = tfidf_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
  print(w, s)
"""
factor=df2["descripcion"].count()

scores=[]
dos_mil_palabras_importantes = []
for doc in range(factor):
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      if not w in dos_mil_palabras_importantes:   
          scores.append(-s)
          dos_mil_palabras_importantes.append(w)  



scores_mas_importantes=[]
heapq.heapify(scores)

#fijarse de tener esta cantidad de palabras sino rompe

for x in range(cantidad_de_palabras_que_quiero):
    scores_mas_importantes.append(heapq.heappop(scores))

dos_mil_palabras_importantes = []    
    
for doc in range(df2["descripcion"].count()):
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      if -s in scores_mas_importantes:   
          dos_mil_palabras_importantes.append(w) 
            
print("ok")  

ok


In [51]:
print(dos_mil_palabras_importantes
    )

['estas', 'interes', 'grup', 'ingres', 'dat', 'vincul', 'industri', 'automotriz', 'estar', 'seleccion', 'candidat', 'desempeñ', 'line', 'produccion', 'plant', 'industrial', 'deber', 'cont', 'secundari', 'complet', 'turn', 'rotat', 'postulat', 'referent', 'eleg', 'pod', 'envi', 'cv', 'mail', 'com', 'ar', 'www', 'trabaj', 'invit', 'form', 'argentin', 'compromis', 'dese', 'person', 'pil', 'clav', 'compañi', 'desarroll', 'profesional', 'colabor', 'bas', 'crecimient', 'sumat', 'contribu', 'fabric', 'alimentici', 'farmaceut', 'are', 'electromecan', 'instal', 'electr', 'interpretacion', 'plan', 'pc', 'prevent', 'correct', 'autonomi', 'inici', 'proactiv', 'cajer', 'requier', 'habil', 'atencion', 'trat', 'client', 'valor', 'perfil', 'resid', 'plat', 'brind', 'ayud', 'comercial', 'rubr', 'edad', 'sex', 'masculin', 'resident', 'aledañ', 'excluyent', 'registr', 'conduc', 'licenci', 'nacional', 'habilit', 'transport', 'carg', 'vigent', 'presenci', 'personal', 'limpiez', 'comprob', 'tim', 'mecan', '

In [33]:
data2=df2["descripcion"][43]
data2#nuevo

'busqued anal credit jr sector entid bancari posicion respons ser analisis evaluacion propuest credit participacion conjunt credit administracion carter requisit postulat si sos profesional estudi avanz carrer cienci econom licenci administracion ingenier ten actitud proactiv manej relacion interpersonal equip orientacion result cont manej paquet offic valor conoc segment experient laboral analisis balanc riesg credit benefici condicion contratacion lun viern jorn tim trabaj microcentr empres ofrec condicion contratacion'

In [34]:
def mas_impor(description):
    
    resultado = []
    
    for palabra in description.lower().split():
        if palabra not in dos_mil_palabras_importantes:
            continue
            
        resultado.append(palabra)
    
    return ' '.join(resultado)

In [35]:
df2["descripcion"]=df2["descripcion"].map(mas_impor)

In [36]:
data2=df2["descripcion"][43]
data2#nuevo

'credit sector entid bancari posicion evaluacion propuest credit conjunt credit postulat si sos profesional licenci actitud proactiv interpersonal cont valor segment balanc credit benefici jorn tim trabaj microcentr'

In [37]:
data1=df2["descripcion"][43]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> similitud:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)
print("ok")

Estimated Jaccard for data1 and data2 is 1.0
credit sector entid bancari posicion evaluacion propuest credit conjunt credit postulat si sos profesional licenci actitud proactiv interpersonal cont valor segment balanc credit benefici jorn tim trabaj microcentr
Estimated Jaccard for data1 and data2 is 0.5859375
cobranz compañi financ grup reconoc entid bancari posicion grup postulat si sos profesional actitud proactiv interpersonal cont valor jorn tim trabaj microcentr
Estimated Jaccard for data1 and data2 is 0.3203125
industrial posicion deseabl cont comprob posicion encontr vigent futur desarroll desarroll plan propuest valor proactiv interpersonal jorn trabaj tim trabaj pod envi cv via
Estimated Jaccard for data1 and data2 is 0.3125
industrial posicion cont comprob posicion encontr vigent futur desarroll distribucion desarroll plan propuest valor proactiv interpersonal jorn trabaj tim trabaj
ok


In [38]:
data1=df2["descripcion"][5]
print(data1)

cajer requier secundari complet habil atencion trat client valor perfil proactiv resid plat


In [39]:
data1=df2["descripcion"][5]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> similitud:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)  
print("ok")

Estimated Jaccard for data1 and data2 is 1.0
cajer requier secundari complet habil atencion trat client valor perfil proactiv resid plat
Estimated Jaccard for data1 and data2 is 0.3046875
caden supermerc cajer repositor cont secundari complet informat valor caj atencion client mercaderi interes cont tim eventual
Estimated Jaccard for data1 and data2 is 0.328125
caden supermerc cajer cont secundari complet sex masculin edad caj sanitari tim valor caj atencion client interes cont tim
Estimated Jaccard for data1 and data2 is 0.3125
tim secundari complet basic electron acces basic contact atencion client habil trat interpersonal negociacion proactiv compromis client
Estimated Jaccard for data1 and data2 is 0.34375
multinacional rubr coordinacion comunic client instalacion carg dat perfil atencion client secundari complet tim client habil empati resolut enfoc proactiv
Estimated Jaccard for data1 and data2 is 0.4609375
desarroll client maquinari comercial desarroll client requier perfil pers

In [44]:
data1=df2["descripcion"][10]
print(data1)

impuest belgran capital federal junior impuest client global capital federal belgran encuentr contador impuest contabl basic imposit form are encuentr impuest nacional informacion impuest basic ingres imposit contabl cont tim trabaj interes profesional global postulat contador resident capital federal edad salari complet salari tim trabaj belgran are impuest interes envi cv com ar referent junior impuest


In [45]:
data1=df2["descripcion"][10]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> similitud:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)
print("ok")

Estimated Jaccard for data1 and data2 is 1.0
impuest belgran capital federal junior impuest client global capital federal belgran encuentr contador impuest contabl basic imposit form are encuentr impuest nacional informacion impuest basic ingres imposit contabl cont tim trabaj interes profesional global postulat contador resident capital federal edad salari complet salari tim trabaj belgran are impuest interes envi cv com ar referent junior impuest
Estimated Jaccard for data1 and data2 is 0.328125
crecimient seleccion contabl contador nacional desempeñ contabl bancari contabl fond caj requier are contabl contabl capital federal trabaj tim interes envi cv
Estimated Jaccard for data1 and data2 is 0.328125
form trabaj trat client intern cont atencion valor are legal cobranz rubr inmobiliari sex edad tim resident capital federal interes envi cv
Estimated Jaccard for data1 and data2 is 0.3359375
nacional crecimient seleccion contabl contador nacional desempeñ bancari contabl requier contabl